# Test Project

<h3>Importing packages<h3>
    

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time

<h3>Tracking performance. Tracking time to scrap each page<h3>

In [3]:
start_time = time.time()
page_counter = 0

<h3>Creating empty lists to store data in<h3>

In [4]:
book_list = []
price_list = []
stock_list = []
star_list = []
num_available = []
category_list = []

<h3>Requesting pages to scrap<h3>

In [5]:
html_text = requests.get('http://books.toscrape.com/catalogue/page-1.html').text
soup = BeautifulSoup(html_text, 'lxml')
button = soup.find('li', class_='next')
pager=button.text

<h3>Wrapping the code in  a function<h3>

In [6]:
def find_books():
    books = soup.find_all('li', class_='col-xs-6 col-sm-4 col-md-3 col-lg-3')    
    for book in books:
        book_title = book.article.h3.a['title']
        book_price_pound = book.article.find('div', class_='product_price').p.text
        book_price =re.findall('£([0-9.0]+)', book_price_pound) 
        avail=book.article.find('div', class_='product_price').find('p', class_='instock availability').text.strip()
        stars = book.article.p['class']
        star = stars[1]
        
        #Storing data in empty lists
        book_list.append(book_title)
        price_list.append(float(book_price[0]))
        stock_list.append(avail)
        star_list.append(star)
        
        #Checking quantity of books available
        book_link = requests.get('http://books.toscrape.com/catalogue/'+book.article.h3.a['href']).text
        url = book_link
        soup2 = BeautifulSoup(url, 'lxml')
        stock_avail = soup2.find('p', class_='instock availability').text.strip()
        
        #Using regular expression to isolate numerical values
        stock_nums = re.findall('[0-9]+', stock_avail)
        stock_num = stock_nums[0]
        num_available.append(stock_num)
        
        #Scraping book catergory
        cnt=0
        category = soup2.find_all('li')
        for i in category:
            cnt+=1
            #Book's category is number 3 on the list
            if cnt ==3:
                category_list.append(i.text.strip())

<h3>A function to run the program as long as there in 'next' page to scrap<h3>

In [ ]:
if __name__ =='__main__':
    while pager == 'next':
        #Tracking Performance
        page_counter+=1
        end_time = time.time()
        seconds = end_time - start_time
        minutes = seconds/60
        print('Page #: ', page_counter)
        print('Minutes: ', minutes)
        
        
        #Requesting multiple pages if available. If no more pages, break
        try:
            find_books()
            button = soup.find('li', class_='next')
            next_page = button.a['href']
            url = 'http://books.toscrape.com/catalogue/'+str(next_page)
            pager=button.text
            html_text = requests.get(url).text
            soup = BeautifulSoup(html_text, 'lxml')
            
            
        except:
            print('Done')
            pager = 'last'
            break

Page #:  1
Minutes:  4.865206849575043
Page #:  2
Minutes:  5.080256946881613
Page #:  3
Minutes:  5.29583603143692
Page #:  4
Minutes:  5.524510331948599
Page #:  5
Minutes:  5.745488500595092
Page #:  6
Minutes:  5.962447468439738
Page #:  7
Minutes:  6.180831968784332
Page #:  8
Minutes:  6.417035349210104
Page #:  9
Minutes:  6.631955254077911
Page #:  10
Minutes:  6.855706652005513


<h3>Merging lists to create a dataframe<h3>

In [ ]:
dataset = pd.DataFrame(list(zip(book_list, category_list,star_list,price_list,stock_list,num_available)), columns = ['Title','Book_category','Star_rating', 'Price', 'Stock', 'Quantity'])
print(dataset)

<h3>Saving my Dataset to csv<h3>

In [ ]:
dataset.to_csv('Link to save a file .csv',sep=',', index=False)